Project: Echoes of Power
=====

In this problem set, you will use iPython notebook and various python tools to read data and analyze it.

We will use data from the paper 

In [1]:
%pylab inline



Populating the interactive namespace from numpy and matplotlib


In [2]:
utterances = {}
def load_data():
    with open('wikipedia.talkpages.conversations.txt', "r") as f:
        for line in f:
            split = line.split("+++$+++")
#             print len(split)
            try:
                utterances[split[0].strip()] = {"user_id": split[1].strip(),
                                        "talkpage_user": split[2].strip(), 
                                        "conversation_root": split[3].strip(),
                                        "reply_to": split[4].strip(),
                                        "timestamp": split[5].strip(),
                                        "timestamp_unix": split[6].strip(),
#                                       looks like readme has a small mistake                                     
                                        "raw_text": split[7].strip(),
                                        "text": split[8].strip(),        
                                        }
            except IndexError:
                pass
load_data()


In [3]:
# utterances.keys()

In [4]:
conversations = {}
for u_id, utter in utterances.iteritems():
    user_id = utter["user_id"]
    reply_id = utter.get("reply_to")
#     print utterances.get(reply_id)
#     print reply_id
    root_conversation = utter["conversation_root"]
   
#   doubt
    if root_conversation == u_id:
        continue
    if conversations.get(root_conversation) == None:
        conversations[root_conversation] = [u_id]
    else:
        conversations[root_conversation].append(u_id)
        
        
#     if reply_id and utterances.get(reply_id):
#         reply_user_id = utterances[reply_id]["user_id"]
#         if(conversations.get((user_id, reply_user_id)) == None):
#             conversations[utter["conversation_root"]] = [reply_id, u_id]
#         else:
#             conversations[(user_id, reply_user_id)].append(reply_id)
#             conversations[(user_id, reply_user_id)].append(u_id)
conversations
print len(conversations.keys())

125292


In [5]:
count = 0.0
for i in conversations.keys():
    if utterances[i]["user_id"] == utterances[i]["talkpage_user"]:
        count += 1
print count/125292

0.0426763081442


In [6]:
cnt = 0
for i in utterances.keys():
    if(utterances[i]["reply_to"] == i):
        cnt += 1
#     print utterances[i]["reply_to"]
#     pass
cnt

0

In [7]:

#'577840': ['577843', '577842', '577841', '577844']]
print utterances['577843']
print (utterances['577842']['reply_to'])
print utterances['577841']
# print utterances['577844']['reply_to']
print utterances['577840']

{'conversation_root': '577840', 'user_id': 'Zanimum', 'talkpage_user': 'Zanimum', 'timestamp_unix': '1.179627540E09', 'reply_to': '577842', 'timestamp': '2007-05-19 19:19:00', 'text': "::: No, I'll keep it. :) Just wanted to make sure you knew I didn't click the camera myself. Anyway, thanks for noticing my efforts.  --  [[User:Zanimum|Zanimum]] 19:19, 19 May 2007 (UTC)", 'raw_text': "No, I'll keep it. :) Just wanted to make sure you knew I didn't click the camera myself. Anyway, thanks for noticing my efforts.  --"}
577841
{'conversation_root': '577840', 'user_id': 'Zanimum', 'talkpage_user': 'Zanimum', 'timestamp_unix': '1.179626880E09', 'reply_to': '577840', 'timestamp': '2007-05-19 19:08:00', 'text': ': [Blush] I don\'t know if I do... they were taken by other volunteer photographers, that I "commissioned". I\'ve been aranging as many photo shoots as possible, to get this type of image.  --  [[User:Zanimum|Zanimum]] 19:08, 19 May 2007 (UTC)', 'raw_text': '[Blush] I don\'t know if I

In [8]:
def get_other_user(conversation_list, root_user):
    for i in conversation_list:
        other_user = utterances[i]["user_id"]
        if utterances[i]["user_id"] != root_user and other_user != "":
            return other_user

In [9]:
# conversations

In [10]:
conv_list = ['445339','445338','445337','445336','445335','445344','445345','445342','445343','445340','445341']
# '228054': ['228055', '228056']
conv_list = ['228054','228055', '228056']
conv_list = ['320591', '320597', '320595', '320594', '320593', '320592', '320596']

class Node:
    def __init__(self, u_id):
        self.u_id = u_id
        self.child = None
        self.parent = None
#     def add_child(self, child_node=None, parent_node=None):
#         self.child = child_node
#         self.parent = parent_node
#         return True


def get_root(node_list, start_node):
    if start_node.parent == None:
        return start_node
    else:
        parent = start_node.parent
        return get_root(node_list, node_list[parent])


def compute_conversation_hierarchy(conv_list):
    node_list = {}
    
    for i in conv_list:
        node_list[i] = Node(i)
        
    for i in conv_list:
        reply_to = utterances[i]["reply_to"]
        if reply_to != '-1' and reply_to != "initial_post":
            node_list[i].parent = reply_to
            node_list[reply_to].child = i
#           doubt : if a message has more than 1 replies then we are ignosing that here
#                 : 
        
        
    root = get_root(node_list, node_list.values()[0])
    new_list = [root.u_id]
    cur = root
    while(len(new_list) < len(node_list) and cur.child != None):
        cur = node_list[cur.child]
        new_list.append(cur.u_id)
        
    return new_list

print compute_conversation_hierarchy(conv_list)

#     for i in node_list.values():
#         print i.u_id
#         print "its child is %s" % i.child

#         print "its parent is %s" % i.parent

['320591', '320592', '320593', '320594', '320595', '320596', '320597']


In [11]:
import copy

final_conversations = {}

user_pairs = {}

# break_counter = 0

for i, j in conversations.iteritems():
     
    # break_counter +=1
    
    # if (break_counter > 5): break;
        
    # print 'i,j=',i,j    
        
    conv_list = copy.deepcopy(j)
    conv_list.insert(0, i)
    conv_hierarchy = compute_conversation_hierarchy(conv_list)
    
    u_index = conv_hierarchy[0]
    
    B_user = utterances[u_index]["talkpage_user"]
    A_user = utterances[u_index]["user_id"]
    
    
    # assumption validation code
    inValid_conv = False
    counter = 0
    
    
    for u_id in conv_hierarchy:
        if ((counter%2 == 0) and (utterances[u_id]['user_id'] != A_user)): 
            inValid_conv = True
            break;
            
        if (((counter%2 != 0)) and (utterances[u_id]['user_id'] != B_user)): 
            inValid_conv = True
            break;
            
        counter +=1
        
        
    if (inValid_conv):
        continue
    
    if ((A_user, B_user)) not in user_pairs:
        user_pairs[(A_user, B_user)] = 1
    
    if final_conversations.get((A_user, B_user)):
        final_conversations[(A_user, B_user)].append(conv_hierarchy)
    else:
        final_conversations[(A_user, B_user)] = [conv_hierarchy]
#     break
    
    
    
    

In [12]:
print '# conversations = ',len(final_conversations.keys())

users = user_pairs.keys()[20:30]

print users[0]

print '------------------------------'


for user_pair in users:
    print user_pair
    for conv in final_conversations[user_pair]:
        for u_id in conv:
            print u_id
            print utterances[u_id]['user_id']
            # print utterances[utterance]['text']
            print '****'

        print '------------------------------------------'    


# conversations =  47602
('Sluzzelin', 'Wavelength')
------------------------------
('Sluzzelin', 'Wavelength')
32501
Sluzzelin
****
32502
Wavelength
****
32503
Sluzzelin
****
------------------------------------------
('Unschool', 'Muchness')
283621
Unschool
****
283622
Muchness
****
------------------------------------------
('RHaworth', 'Gronky')
36730
RHaworth
****
36731
Gronky
****
------------------------------------------
('MichaelQSchmidt', 'Hullaballoo Wolfowitz')
61719
MichaelQSchmidt
****
61720
Hullaballoo Wolfowitz
****
------------------------------------------
('MER-C', 'Jjron')
331690
MER-C
****
331691
Jjron
****
------------------------------------------
331777
MER-C
****
------------------------------------------
331800
MER-C
****
331801
Jjron
****
------------------------------------------
331649
MER-C
****
------------------------------------------
331818
MER-C
****
331819
Jjron
****
------------------------------------------
331785
MER-C
****
331786
Jjron
****
-----

In [ ]:
def append_to_left_of(item, li, item_to_insert, conv_list):
    if(item in li):
        index= li.index(item)
        li.insert(index, item_to_insert)
        conv_list.remove

def make_conv_list(conv_list):
    cur_conv = [conv_list[0]]
    for c in range(1,len(conv_list)):
        reply_to = utterances[c]["reply_to"]
        

conv_list = ['445339','445338','445337','445336','445335','445344','445345','445342','445343','445340','445341']

def get_ordered_conversation(conv_list):
    temp = []
    while(len(conv_list) > 0):
        conv_list = []
        make_conv_list(conv_list)
    
    for c in conv_list:
        reply_to = utterances[c]["reply_to"]
        append_to_left_of
        print reply_to
        
        
get_ordered_conversation(conv_list)

In [49]:
conv_left = []


In [ ]:
conversation_threads = {}
for i, v in conversations.iteritems():
    root_user = utterances[i]["user_id"]
    other_user = get_other_user(v, root_user)
    conversation_threads[(root_user, other_user)] = get_ordered_conversation(v)
conversation_threads

In [13]:
import nltk

from nltk.stem.porter import *
from nltk import sent_tokenize,word_tokenize,porter
from nltk.corpus import brown
from nltk.corpus import ptb
from nltk.corpus import treebank

import csv
from collections import Counter, defaultdict


In [ ]:
nltk.download()

In [14]:
def word_proc(x, preProcessing_type):
    
    if (preProcessing_type == 0): return x
    if (preProcessing_type == 1): return x.lower()
    if (preProcessing_type == 2): 
        stemmer = PorterStemmer()
        x = x.lower()
        return stemmer.stem(x)

    return x # default 

def create_markers(category):

    file = open(category+'.txt', "r")
    markers = file.read()
    
    tokens = (re.split('[\t\n]', markers.strip()))
            
    return tokens


conjunction_list = create_markers('conjunctions')  

print conjunction_list

['after', 'although', 'and', 'as', 'as far as', 'as how', 'as if', 'as long as', 'as soon as', 'as though', 'as well as', 'because', 'before', 'both', 'but', 'either', 'even if', 'even though', 'for', 'how', 'however', 'if', 'if only', 'in case', 'in order that', 'neither', 'nor', 'now', 'once', 'only', 'or', 'provided', 'rather than', 'since', 'so', 'so that', 'than', 'that', 'though', 'till', 'unless', 'until', 'when', 'whenever', 'where', 'whereas', 'wherever', 'whether', 'while', 'yet']


In [15]:
marker_list = {}
utterances_tokenized = {}

""" 
for consistency with prior work, we employed eight of the nine
LIWC-derived categories [41] deemed to be processed by humans
in a generally non-conscious fashion [28]. Our eight markers are
thus: articles, auxiliary verbs, conjunctions, high-frequency ad-
verbs, impersonal pronouns, personal pronouns, prepositions, and
quantifiers (451 lexemes total)
"""    

"""
I suggest using POS-labeled data to compute the most frequent words in each category, 
and then just make a cutoff. Most of these categories are closed-class, so your cutoff can be zero,
except for the adverbs.

Tagged data is available with NLTK.

I think the Brown corpus tagset distinguishes quantifiers and the PTB doesn't, 
but I'm not sure; you can check. NLTK should give access to a small subset of these corpora, 
but it should be enough for you to build a dictionary. articles are sometimes called "Determiners" 
in these sorts of corpora.

"""

tag_lists = {}
tag_counter = Counter()
adverb_word_counter = Counter()

def populateMarkers():
    
    print 'Brown corpus'
    # print brown.words()[0:10]
    # print brown.tagged_paras()[0:2]
    
    tagged_words = treebank.tagged_words()
    

    
    # print '#words',len(tagged_words)
    
    for pair in tagged_words:
        tag  = pair[1]
        word = word_proc(pair[0], 1)
        
        if tag not in tag_lists:
            tag_lists[tag] = []
        
        if (word not in tag_lists[tag]):
            tag_lists[tag].append(word)
    
        if ((tag == 'RB') or (tag == 'RBR') or (tag == 'RBS') or (tag == 'WRB')):
            adverb_word_counter[word] +=1
    
    
    tagged_words = brown.tagged_words()
    tag_lists['QUANT'] = []
    # print '#words',len(tagged_words)
    
    for pair in tagged_words:
        tag  = pair[1]
        word = word_proc(pair[0], 1)
        
        if ((tag == 'ABN') or (tag == 'ABX')):
            if (word not in tag_lists['QUANT']):
                tag_lists['QUANT'].append(word)
    
    
populateMarkers()    

Brown corpus


In [16]:
marker_list['articles'] =  tag_lists['DT']
marker_list['personal_pronouns'] = tag_lists['PRP']
marker_list['impersonal_pronouns'] = tag_lists['PRP$'] + tag_lists['WP'] + tag_lists['WP$']
marker_list['conjunctions'] = tag_lists['CC'] + tag_lists['IN']
marker_list['prepositions'] = tag_lists['IN']
marker_list['auxillary_verbs'] = tag_lists['MD']
marker_list['quantifiers'] = tag_lists['QUANT']

#  for now considering 100 most commonly occurring adverbs
high_freq_adverb_counts = adverb_word_counter.most_common(100)

marker_list['adverbs'] = []

for adverb_count_pair in high_freq_adverb_counts:
    marker_list['adverbs'].append(adverb_count_pair[0])


In [17]:
# code to compute coordination between user a and user b

"""('Tpbradbury', 'Karanacs'): [['357026', '357027', '357028', '357029']], 
    ('Skope', 'Jedi6'):         [['500004', '500005'], ['375918', '375919'], ['500181', '500183'], 
                                 ['500029', '500030', '500031', '500032']]
"""
"""

utterances = {}
                utterances[split[0].strip()] = {"user_id": split[1].strip(),
                                        "talkpage_user": split[2].strip(), 
                                        "conversation_root": split[3].strip(),
                                        "reply_to": split[4].strip(),
                                        "timestamp": split[5].strip(),
                                        "timestamp_unix": split[6].strip(),
#                                       looks like readme has a small mistake                                     
                                        "raw_text": split[7].strip(),
                                        "text": split[8].strip(),        
                                        
"""



def tokenize(utterance):
    preProcessing_type = 1
    
    text = utterances[utterance]["text"]
    
    word_list = (text.strip()).split(' ')
    
    tokens = Counter()
    
    
    # print 'text',text
    
    # for word in word_tokenize(text):
    for word in word_list:
        tokens[word_proc(word, preProcessing_type)] += 1

    return tokens


def contains(utterance, marker):
    
    preProcessing_type = 1
    
    if (utterance not in utterances_tokenized):
        utterances_tokenized[utterance] = tokenize(utterance) # returns dictionary with keys as tokens
        
    # print utterances_tokenized

    # print "--------------------------------------"
    
    word_list = marker_list[marker]
    
    for i in range(len(word_list)):
        word = word_proc(word_list[i], preProcessing_type)
        if (word in utterances_tokenized[utterance]):
            return True
    
    return False






# input: list of utterance list. Each utterance list is a conversation   
def compute_coordination(conv_list):
    
    # marker_list['articles'] =  ['a', 'an', 'the']
    # marker_list['personal_pronouns'] = ['i','them','her','you','he','she','it','they','we']
    # marker_list['conjunctions'] = create_markers('conjunctions')
    
    
    coordination_list = {}
    
    for marker in marker_list.keys():
        
        a_usage_count = 0
        b_usage_count = 0
        
        b_follows_a_count = 0
        b_utterance_count = 0
        
        for conversation in conv_list:
            for i in range(len(conversation)): #['500029', '500030', '500031', '500032']
                
                # print 'i',i
                
                if (i%2 != 0):continue # for skipping B's utterance
                    
                 
                a_utterance = conversation[i]
                b_utterance = None
                if (i+1 < len(conversation)):
                        b_utterance = conversation[i+1]
                        b_utterance_count +=1
            
                a_utterance_has_m = contains(a_utterance, marker)
                
                b_utterance_has_m = False
                if (b_utterance != None):
                    b_utterance_has_m = contains(b_utterance, marker)
                
                if (a_utterance_has_m):
                    a_usage_count +=1
                    
                if (b_utterance_has_m):    
                    b_usage_count +=1
                    
                if (a_utterance_has_m and b_utterance_has_m):
                    b_follows_a_count +=1
       
    
#        print 'marker',marker
#        print 'b_follows_a_count',b_follows_a_count
#        print 'a_usage_count',a_usage_count 
#        print  'b_usage_count',b_usage_count
#        print 'b_utterance_count',b_utterance_count 
#        print '----------------------'
        if (a_usage_count == 0 or b_utterance_count == 0): 
            coordination = -10 # invalid
        else:
            coordination = 1.0*b_follows_a_count/a_usage_count - 1.0*b_usage_count/b_utterance_count
        
#       if ((a_usage_count != 0) and (b_utterance_count !=0) ): 
#            print 'follows',1.0*b_follows_a_count/a_usage_count
#            print 'base rate',1.0*b_usage_count/b_utterance_count
        
        coordination_list[marker] = coordination
                # if ()
    
    return coordination_list
   

# test code : tested manually    
# print compute_coordination(final_conversations[('Skope', 'Jedi6')])

# for conv in final_conversations[('Skope', 'Jedi6')]:
#    for utterance in conv:
#        print utterances[utterance]['user_id']
#        print utterances[utterance]['text']
#        print '****'
        
#    print '----------------------'    
    
#        
# test code : -10 indicates cannot be computed 
i = 0
for key in final_conversations.keys():    
    i +=1
    if (i > 20): break
    # print 'conv_list',final_conversations[key]
    
    print '------------------------------------------'
    print compute_coordination(final_conversations[key])
    


------------------------------------------
{'articles': 0.0, 'impersonal_pronouns': 0.0, 'quantifiers': -10, 'conjunctions': 0.0, 'prepositions': 0.0, 'personal_pronouns': 0.0, 'adverbs': 0.0, 'auxillary_verbs': 0.0}
------------------------------------------
{'articles': -10, 'impersonal_pronouns': -10, 'quantifiers': -10, 'conjunctions': -10, 'prepositions': -10, 'personal_pronouns': -10, 'adverbs': -10, 'auxillary_verbs': -10}
------------------------------------------
{'articles': -10, 'impersonal_pronouns': 0.0, 'quantifiers': -10, 'conjunctions': 0.0, 'prepositions': 0.0, 'personal_pronouns': -10, 'adverbs': 0.0, 'auxillary_verbs': -10}
------------------------------------------
{'articles': -10, 'impersonal_pronouns': -10, 'quantifiers': -10, 'conjunctions': -10, 'prepositions': -10, 'personal_pronouns': -10, 'adverbs': -10, 'auxillary_verbs': -10}
------------------------------------------
{'articles': 0.0, 'impersonal_pronouns': 0.0, 'quantifiers': -10, 'conjunctions': 0.0, 'p

Parul's code

In [54]:
import numpy as np

In [62]:
admin_data = {}
admin_file = open("wikipedia.talkpages.admins.txt", 'r')
for line in admin_file:
    cols = line.split(" ")
    if len(cols) == 2:
        admin_data[cols[0]] = cols[1].split("\n")[0]
    else:
        name = ""
        for i in range(len(cols)-2):
            name += cols[i]+" "
        name+= cols[i+1]
        admin_data[name] = cols[len(cols)-1].split("\n")[0]
#print admin_data

In [63]:
user_data = {}
user_file = open("wikipedia.talkpages.userinfo.txt", 'r')
for line in user_file:
    cols = line.split(" +++$+++ ")
    user_data[cols[0]] = cols[2]#.split("\n")[0]
print len(user_data)

26397


In [64]:
def aggregate1(cbA):
    total = 0
    i = 0
    #print cbA
    for coord in cbA.values():
        c_b_a = coord.values()
        if -10 not in c_b_a:
            i+=1
            total += np.average(c_b_a)
    CBA = total/i
    return CBA

In [65]:
def aggregate2(cbA):
    total = np.array([0.0,0.0,0.0]) #,0.0,0.0,0.0,0.0,0.0])
    counts = np.zeros((1,3))
    i = 0
    for coord in cbA.values():
        c_b_a = coord.values()
        np_cba = np.array(c_b_a)
        total[np_cba != -10] += np_cba[np_cba != -10]
        #print (np_cba != 0).astype(int)
        #print np_cba[np_cba != 0]
        counts += (np_cba != -10).astype(int)
    #print counts[counts != 0]
    #print total[total != 0]
    if 0 not in counts:
        CBA = np.average(total/counts)
    else:
        CBA = np.sum(total[total != -10]/counts[counts != -10])/3#8
    return CBA


In [66]:
def aggregate3(cbA):
    total = 0
    i = 0
    for coord in cbA.values():
        c_b_a = coord.values()
        np_cba = np.array(c_b_a)
        #print np_cba[np_cba != -10]
        if np_cba[np_cba != -10] != []:
            total += np.average(np_cba[np_cba != -10])
        
    #print len(cbA)
    #print total
    CBA = total/len(cbA)
    return CBA

In [67]:
a_admin = 0
a_notadmin = 0

print len(final_conversations)

for (a,b), conversation in final_conversations.items():
    if a in admin_data:
        a_admin+=1
    else: 
        a_notadmin+=1
        
print a_admin/float(len(final_conversations))
print a_notadmin/float(len(final_conversations))


82801
0.249127425997
0.750872574003


In [68]:
def getConversations(user):
    all_converation = {'admin':[], 'non_admin': [], 'U': []}
    for (a,b), conversation in final_conversations.items():
        if b == user:
            if a in admin_data:
                all_converation['admin'] += conversation
            elif a not in admin_data:
                all_converation['non_admin'] += conversation
            all_converation['U'] += conversation
    return all_converation

In [72]:
#Construct C(U,G_high)
C_U_G_high = {}
C_U_G_low = {}
C_G_high_U = {}
C_G_low_U = {}

i = 0
for (a,b), conversation in final_conversations.items():
    if i > 1000:
        break
    
    
    if ("admin",b) not in C_U_G_high and ("non_admin",b) not in C_U_G_low: 
        all_conversation = getConversations(b)
        i+=1
        
    if a in admin_data:
        if ("admin",b) not in C_U_G_high:
            C_U_G_high[("admin",b)]= compute_coordination(all_conversation['admin'])
        '''  
        new_conversation = []
        for thread in conversation:
            thread = thread[1:len(thread)]
            new_conversation.append(thread)
        C_G_high_U[(b,a)]= compute_coordination(new_conversation)
        '''

    elif a not in admin_data:
        if ("non_admin",b) not in C_U_G_low:
            C_U_G_low[("non_admin",b)]= compute_coordination(all_conversation['non_admin'])
        '''
        new_conversation = []
        for thread in conversation:
            thread = thread[1:len(thread)]
            new_conversation.append(thread)
        C_G_low_U[(b,a)]= compute_coordination(new_conversation)
        
        '''
        
    if b in admin_data:
        if ("U",b) not in C_G_high_U:
            C_G_high_U[("U",b)]= compute_coordination(all_conversation['U'])
        '''
        new_conversation = []
        for thread in conversation:
            thread = thread[1:len(thread)]
            new_conversation.append(thread)
        C_U_G_high[(b,a)]= compute_coordination(new_conversation)
        '''
    
    elif b not in admin_data:
        if ("U",b) not in C_G_low_U:
            C_G_low_U[("U",b)]= compute_coordination(all_conversation['U'])
        '''
        new_conversation = []
        for thread in conversation:
            thread = thread[1:len(thread)]
            new_conversation.append(thread)
        C_U_G_low[(b,a)]= compute_coordination(new_conversation)
        '''


In [70]:
groups = ['C_U_G_high','C_U_G_low','C_G_high_U','C_G_low_U']
group_results = [C_U_G_high, C_U_G_low , C_G_high_U , C_G_low_U]

print len(C_U_G_high)

agg1 = {}
agg2 = {}
agg3 = {}

for i in range(len(groups)):
    agg1[groups[i]] = aggregate1(group_results[i])
    agg2[groups[i]] = aggregate2(group_results[i])
    agg3[groups[i]] = aggregate3(group_results[i])
    
print agg1
print agg2
print agg3

18
{'C_U_G_high': -0.1160971671903919, 'C_U_G_low': -0.10927522027035985, 'C_G_high_U': -0.11591807480568213, 'C_G_low_U': -0.1410598476165344}
{'C_U_G_high': -0.11609716719039191, 'C_U_G_low': -0.10927522027035985, 'C_G_high_U': -0.11591807480568207, 'C_G_low_U': -0.14105984761653437}
{'C_U_G_high': -0.10964732456870346, 'C_U_G_low': -0.10927522027035985, 'C_G_high_U': -0.11591807480568213, 'C_G_low_U': -0.1410598476165344}
